In [59]:
from json_extractor import from_file_get_n_docs
import json
from token_list_mapper import map_token_list_a_to_b
from similarity import similar

# Get Parsed docs

In [2]:
STANFORD_FILENAME = '100_parsed_from_stanford.jsonl'
TAGME_FILENAME    = '100_parsed_from_tagme.jsonl'
WIKIFIER_FILENAME = '100_parsed_from_wikifier.jsonl'

In [3]:
stanford_parsed_docs = from_file_get_n_docs(STANFORD_FILENAME, 5)

In [4]:
wikifier_parsed_docs = from_file_get_n_docs(WIKIFIER_FILENAME, 5)

In [331]:
tagme_parsed_docs    = from_file_get_n_docs(TAGME_FILENAME, 5)

In [332]:
test_index = 0

In [333]:
stanford_doc = stanford_parsed_docs[test_index]

In [334]:
wikifier_doc = wikifier_parsed_docs[test_index]

In [335]:
tagme_doc = tagme_parsed_docs[test_index]

# Generate Word Dicts

## Common token list + tagged-words dictionary => tagged-words list

In [292]:
filter_nones = lambda p: [v for v in enumerate(p) if v[1] is not None]
relative_inverse_score = lambda test_word_index, compared_index: 1.0 / float(abs(test_word_index - compared_index) + 1)
similar_in_list = lambda w, wi, l: [(similar(w, lw, None) + relative_inverse_score(wi, i), (similar(w, lw, None), relative_inverse_score(wi, i), i, lw) for i, lw in enumerate(l) if similar(w, lw, 0.6)]

max_or_none = lambda l: max(l) if l else None
max_similar_in_list = lambda w, wi, l: (w, max_or_none(similar_in_list(w, wi, l)))

new_mapping_test = lambda a_tokens, b_tokens: filter(lambda match: match[1][1] and match[1][1][0] >= 0.6, map(lambda (i, w): (i, max_similar_in_list(w, i, b_tokens)), enumerate(a_tokens)))
new_mapping = lambda mapping: {m[0]: m[1][1][2] for m in mapping}

SyntaxError: invalid syntax (<ipython-input-292-bbe9296a3d80>, line 3)

In [291]:
def map_parsed_doc_to_token_list(doc, token_list):
    token_list_to_doc = new_mapping(new_mapping_test(token_list, doc['tokens']))
    parsed_token_list = map(
        lambda (i, _): doc['tagged-words'].get(str(token_list_to_doc.get(i, -1)), None),
        enumerate(token_list)
    )
    return parsed_token_list

doc = tagme_doc
mapped = map_parsed_doc_to_token_list(doc, stanford_doc['tokens'])
# filter_nones(map_parsed_doc_to_token_list(stanford_doc, stanford_doc['tokens'])).__len__(), stanford_doc['tagged-words'].keys().__len__()
# filter_nones(map_parsed_doc_to_token_list(tagme_doc, stanford_doc['tokens'])).__len__(), tagme_doc['tagged-words'].keys().__len__()
# map(lambda tag: (tag[1], stanford_doc['tokens'][tag[0]]), filter_nones(map_parsed_doc_to_token_list(tagme_doc, stanford_doc['tokens'])))

TypeError: list indices must be integers, not str

In [277]:
filter_nones(mapped).__len__(), doc['tagged-words'].keys().__len__()

(22, 137)

### Issue mapping from stanford to tagme.
See all the tagged-words that are not in the mapped list

In [278]:
doc_to_token_list = map_token_list_a_to_b(doc['tokens'], stanford_doc['tokens'])
[(doc['tokens'][int(k)], v) for k, v in doc['tagged-words'].items() if int(k) not in doc_to_token_list]

[(u'Share',
  {u'dbpedia_categories': [u'Equity securities', u'Stock market'],
   u'end': 133,
   u'id': 867877,
   u'link_probability': u'0.00434',
   u'rho': u'0.04382',
   u'spot': u'Share',
   u'start': 133,
   u'title': u'Share (finance)'}),
 (u'area',
  {u'dbpedia_categories': [u'Area'],
   u'end': 132,
   u'id': 1209,
   u'link_probability': u'0.00662',
   u'rho': u'0.02101',
   u'spot': u'area',
   u'start': 132,
   u'title': u'Area'}),
 (u'designated',
  {u'dbpedia_categories': [u'British military aircraft',
    u'Military aircraft designation systems'],
   u'end': 131,
   u'id': 75085,
   u'link_probability': u'0.00561',
   u'rho': u'0.01074',
   u'spot': u'designated',
   u'start': 131,
   u'title': u'British military aircraft designation systems'}),
 (u'drink',
  {u'dbpedia_categories': [u'Foods', u'Food and drink', u'Cuisine'],
   u'end': 130,
   u'id': 10646,
   u'link_probability': u'0.02989',
   u'rho': u'0.07839',
   u'spot': u'food and drink',
   u'start': 128,
   u't

In [263]:
doc_to_token_list

{0: 0,
 1: 1,
 2: 2,
 3: 5,
 4: 6,
 5: 10,
 6: 12,
 7: 13,
 8: 21,
 9: 22,
 10: 24,
 11: 27,
 12: 28,
 13: 30,
 14: 31,
 15: 33,
 26: 48,
 31: 53,
 53: 75,
 68: 90,
 74: 97}

In [264]:
[ (doc['tokens'][d] , stanford_doc['tokens'][s]) for d, s in doc_to_token_list.items() ]

[(u'VETERANS', u'VETERANS'),
 (u'saluted', u'saluted'),
 (u'Worcester', u"Worcester's"),
 (u'breakfast', u'breakfast'),
 (u'club', u'club'),
 (u'won', u'won'),
 (u'hearts,', u'hearts,'),
 (u'minds', u'minds'),
 (u'HM', u'HM'),
 (u'Forces', u'Forces'),
 (u'met', u'met'),
 (u'Postal', u'Postal'),
 (u'Order', u'Order'),
 (u'Foregate', u'Foregate'),
 (u'Street', u'Street'),
 (u'10am', u'10am'),
 (u'month', u'other'),
 (u'well', u'we'),
 (u'set', u'eat'),
 (u'of', u'of'),
 (u'can', u'and')]

In [353]:
def inverse_scoring(a_i, b_i):
    score = 1.0 / (float(abs(a_i - b_i) + 0.1 ))
    return score

# change a_index for a relative position in its list
def elements_similarity(a_i, a, a_len, b_i, b, b_len):
    return (
        inverse_scoring(float(a_i)/float(a_len),
                        float(b_i)/float(b_len)) * similar(a, b, None), 
        similar(a, b, None),
        a_i,
        a, 
        b_i,
        b
    )

def similar_to_a_in_list(a_index, a_val, a_list, l):
    return max(
        map (
            lambda (b_i, b): elements_similarity(a_index, a_val, len(a_list), b_i, b, len(l)),
            enumerate(l)
        )
    )

def map_list_a_to_b(a, b):
    return [similar_to_a_in_list(i, tok, a, b) for (i, tok) in enumerate(a)]

In [354]:
mapping_test = map_list_a_to_b(tagme_doc['tokens'], stanford_doc['tokens'])

In [355]:
stanford_doc['tokens'][80:100], tagme_doc['tokens'][80:100]

([u'country.',
  u'He',
  u'said:',
  u'"As',
  u'you',
  u'can',
  u'see',
  u'from',
  u'the',
  u'picture,',
  u'we',
  u'had',
  u'a',
  u'good',
  u'response.',
  u'Five',
  u'out',
  u'of',
  u'the',
  u'10'],
 [u'country.',
  u'He',
  u'said:',
  u'"As',
  u'you',
  u'can',
  u'see',
  u'from',
  u'the',
  u'picture,',
  u'we',
  u'had',
  u'a',
  u'good',
  u'response.',
  u'Five',
  u'out',
  u'of',
  u'the',
  u'10'])

In [356]:
mapping_test

[(10.0, 1.0, 0, u'VETERANS', 0, u'VETERANS'),
 (10.0, 1.0, 1, u'saluted', 1, u'saluted'),
 (10.0, 1.0, 2, u"Worcester's", 2, u"Worcester's"),
 (10.0, 1.0, 3, u'first', 3, u'first'),
 (10.0, 1.0, 4, u'ever', 4, u'ever'),
 (10.0, 1.0, 5, u'breakfast', 5, u'breakfast'),
 (10.0, 1.0, 6, u'club', 6, u'club'),
 (10.0, 1.0, 7, u'for', 7, u'for'),
 (10.0, 1.0, 8, u'ex-soldiers', 8, u'ex-soldiers'),
 (10.0, 1.0, 9, u'which', 9, u'which'),
 (10.0, 1.0, 10, u'won', 10, u'won'),
 (10.0, 1.0, 11, u'over', 11, u'over'),
 (10.0, 1.0, 12, u'hearts,', 12, u'hearts,'),
 (10.0, 1.0, 13, u'minds', 13, u'minds'),
 (10.0, 1.0, 14, u'and', 14, u'and'),
 (10.0, 1.0, 15, u'bellies.', 15, u'bellies.'),
 (10.0, 1.0, 16, u'The', 16, u'The'),
 (10.0, 1.0, 17, u'Worcester', 17, u'Worcester'),
 (10.0, 1.0, 18, u'Breakfast', 18, u'Breakfast'),
 (10.0, 1.0, 19, u'Club', 19, u'Club'),
 (10.0, 1.0, 20, u'for', 20, u'for'),
 (10.0, 1.0, 21, u'HM', 21, u'HM'),
 (10.0, 1.0, 22, u'Forces', 22, u'Forces'),
 (10.0, 1.0, 23, u

In [358]:
a = ['perro', 'perro"s', '....']
b = ['asdfasda', 'per', 'b', 'perro', 'perros', 'l']
print similar('perro"s',  'perro', None), similar('perro"s', 'perros', None)
map_list_a_to_b(a, b)

0.833333333333 0.923076923077


[(2.8125, 0.75, 0, 'perro', 1, 'per'),
 (3.1249999999999996, 0.8333333333333334, 1, 'perro"s', 3, 'perro'),
 (0.0, 0.0, 2, '....', 5, 'l')]